In [2]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "standardTesting"
FINAL_DATA_NAME = 'standardTesting-full-wavelet'
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "full-standardTesting-gray"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','standardTesting')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [5]:
file_names = [
    filename
    for filename in os.listdir(os.path.join(data_dir, f"{RAW_DATA_SUFFIX}"))
    if not filename.startswith(".")
]

file_list = [
    os.path.join(data_dir, f"toy-{RAW_DATA_SUFFIX}", filename)
    for filename in file_names
]

file_names[:5]


['barbara.npz', 'cameraman.npz', 'house.npz', 'jetplane.npz', 'lake.npz']

In [7]:
'''Assuming No batching is required. Not applicable for agriVision'''

data_dir = os.path.join(ROOT_DIR, "raw-data", "standardTesting", "full-standardTesting-gray")

for channel in ['gray']:

    channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True, image_opener=npz_opener)
    pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-standardTesting-gray-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_fr['band'])
    group_data_map = dict()
    group_data_map_size = dict()
    for group in np.arange(min_group, max_group + 1):
        data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
        group_data_map_size[group] = data.size
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

  0%|          | 0/10 [00:00<?, ?it/s]

Recursion Depth Exceeded Endpoints are 0.0 and 0.021836601342771385
Recursion Depth Exceeded Endpoints are 0.02209708691207961 and 0.043891025496570764
Recursion Depth Exceeded Endpoints are 0.044064508487025304 and 0.06588721794309765
Recursion Depth Exceeded Endpoints are 0.06608954810815872 and 0.08797738773090007
Recursion Depth Exceeded Endpoints are 0.08808572096702875 and 0.11000099875256815
Recursion Depth Exceeded Endpoints are 0.11007033440316696 and 0.1320347170402779
[0.02194551 0.04398869 0.06602095 0.08802615 0.11006022 0.13206901
 0.17606856 0.22015294 0.26415428 0.30814661 0.35217232 0.39618053
 0.44020209 0.52825984]


  0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [8]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


In [9]:
import numpy as np

def npz_opener_pickle(path):
    image = Image.open(path).convert('RGB')
    arr = np.array(image).astype(np.float32)
    jitter = np.random.uniform(-0.5, 0.5, arr.shape)
    arr += jitter
    arr = np.clip(arr, 0, 255)
    return arr

# Wavelet

In [12]:
FINAL_DATA_NAME = 'standardTesting-full-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "standardTesting", f"{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [ ]:
import gc

TRANSFORM = "wavelet"
channel = "red"

channel_wv_full = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
channel_wv_full['data'] = channel_wv_full['data'].apply(lambda x: x.astype(np.float32))  # if needed, or skip if you jittered already

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = channel_wv_full[channel_wv_full['orientation'] == orientation_code].copy()

    if SAVE_DF:
        df_save_path = os.path.join(
            ROOT_DIR,
            "transformed-data",
            f"dataframes/{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"
        )
        pd.to_pickle(channel_wv, df_save_path)

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = {}
    group_data_map_size = {}

    for group in np.arange(min_group, max_group + 1):
        filtered = channel_wv[channel_wv['layer'] == group]
        if filtered.empty:
            continue

        data = filtered['data'].iloc[0]
        sampled = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]

        group_data_map[group] = sampled
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-indoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    freq_df_save_path = os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")
    freq_df.to_csv(freq_df_save_path)

    group_save_base = os.path.join(
        ROOT_DIR,
        "transformed-data",
        f"{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}"
    )
    pd.to_pickle(group_data_map, f"{group_save_base}.pickle")
    pd.to_pickle(group_data_map_size, f"{group_save_base}-size.pickle")

    del channel_wv
    del group_data_map
    del group_data_map_size
    gc.collect()

del channel_wv_full
gc.collect()

9 layers being used


  0%|          | 0/1604 [00:00<?, ?it/s]

0

In [ ]:
import gc

TRANSFORM = "wavelet"
channel = "green"

channel_wv_full = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
channel_wv_full['data'] = channel_wv_full['data'].apply(lambda x: x.astype(np.float32))  # if needed, or skip if you jittered already

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = channel_wv_full[channel_wv_full['orientation'] == orientation_code].copy()

    if SAVE_DF:
        df_save_path = os.path.join(
            ROOT_DIR,
            "transformed-data",
            f"dataframes/{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"
        )
        pd.to_pickle(channel_wv, df_save_path)

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = {}
    group_data_map_size = {}

    for group in np.arange(min_group, max_group + 1):
        filtered = channel_wv[channel_wv['layer'] == group]
        if filtered.empty:
            continue

        data = filtered['data'].iloc[0]
        sampled = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]

        group_data_map[group] = sampled
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-indoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    freq_df_save_path = os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")
    freq_df.to_csv(freq_df_save_path)

    group_save_base = os.path.join(
        ROOT_DIR,
        "transformed-data",
        f"{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}"
    )
    pd.to_pickle(group_data_map, f"{group_save_base}.pickle")
    pd.to_pickle(group_data_map_size, f"{group_save_base}-size.pickle")

    del channel_wv
    del group_data_map
    del group_data_map_size
    gc.collect()

del channel_wv_full
gc.collect()

9 layers being used


  0%|          | 0/1604 [00:00<?, ?it/s]

0

In [ ]:
TRANSFORM = "wavelet"
channel = "blue"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener_pickle)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-indoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/1604 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-18656.088, -13917.058, -13473.333, ...,  18541.072,  18542.385,
        19473.973], shape=(1604,), dtype=float32), np.int64(3): array([-11489.305, -10831.345, -10765.515, ...,  10922.157,  11048.591,
        11585.475], shape=(6416,), dtype=float32), np.int64(4): array([-6686.0264, -6631.2876, -6623.013 , ...,  6230.6436,  7057.608 ,
        7155.0044], shape=(25664,), dtype=float32), np.int64(5): array([-3415.0684, -3306.3215, -3272.6975, ...,  3212.463 ,  3222.9172,
        3244.3333], shape=(100000,), dtype=float32), np.int64(6): array([-1752.1951, -1712.0693, -1647.8092, ...,  1586.6711,  1642.568 ,
        1718.9742], shape=(100000,), dtype=float32), np.int64(7): array([-904.8463 , -814.9612 , -786.8406 , ...,  777.67   ,  811.8638 ,
        889.41144], shape=(100000,), dtype=float32), np.int64(8): array([-468.88113, -414.365  , -398.99982, ...,  392.37805,  408.50107,
        484.63342], shape=(100000,), dtype=float32), np.int64(9): array([-250.79

  0%|          | 0/1604 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-18437.377, -15845.146, -15433.076, ...,  15697.648,  16373.958,
        19876.428], shape=(1604,), dtype=float32), np.int64(3): array([-12940.436, -12440.522, -11031.544, ...,   9259.625,   9426.131,
         9618.772], shape=(6416,), dtype=float32), np.int64(4): array([-6277.2563, -5966.409 , -5947.389 , ...,  5699.646 ,  6210.4897,
        6453.772 ], shape=(25664,), dtype=float32), np.int64(5): array([-3379.3005, -3200.5396, -3187.8923, ...,  3353.6882,  3424.1716,
        3624.729 ], shape=(100000,), dtype=float32), np.int64(6): array([-1765.1881, -1690.8456, -1643.9956, ...,  1623.593 ,  1659.1816,
        1802.0055], shape=(100000,), dtype=float32), np.int64(7): array([-896.9412 , -812.8974 , -785.7233 , ...,  784.42816,  808.7185 ,
        917.4385 ], shape=(100000,), dtype=float32), np.int64(8): array([-461.11572, -411.02817, -393.9123 , ...,  396.0226 ,  411.9547 ,
        472.73605], shape=(100000,), dtype=float32), np.int64(9): array([-252.16

In [14]:
TRANSFORM = "wavelet"
channel = "gray"

for orientation_label in ['horizontal', 'vertical']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else 'V'

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=npz_opener)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-indoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


10 layers being used


  0%|          | 0/10 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-72.27036, 122.7112 , 169.47556, 187.95424, 310.08948, 359.9046 ,
       360.82123, 379.31927, 496.12363, 619.7858 ], dtype=float32), np.int64(3): array([-4.30753326e+02, -3.27243378e+02, -2.37721085e+02, -2.31183136e+02,
       -2.00812637e+02, -1.70977219e+02, -1.59084259e+02, -1.52650162e+02,
       -1.41276443e+02, -1.40082062e+02, -1.38708298e+02, -1.14032265e+02,
       -9.80606537e+01, -9.23010559e+01, -9.09599609e+01, -8.59291916e+01,
       -6.21517944e+01, -5.62970924e+01, -4.54436646e+01, -3.13602867e+01,
       -4.89986944e+00, -2.99207520e+00,  1.46513253e-01,  8.86696529e+00,
        1.24871330e+01,  1.95676346e+01,  3.37049141e+01,  5.53004265e+01,
        6.28108978e+01,  6.79792328e+01,  8.15993500e+01,  8.35013123e+01,
        1.13450790e+02,  1.47129059e+02,  1.82680176e+02,  1.85915955e+02,
        2.24158615e+02,  2.52767410e+02,  5.52967041e+02,  6.19539612e+02],
      dtype=float32), np.int64(4): array([-2.29815186e+02, -2.12266479

  0%|          | 0/10 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-569.5673  , -394.34738 , -288.64337 , -124.9526  ,  -55.56182 ,
        -13.859238,  -12.53167 ,  224.40918 ,  264.36673 ,  596.5461  ],
      dtype=float32), np.int64(3): array([-704.76013 , -193.97209 , -178.26271 , -175.52475 , -148.02417 ,
       -147.35902 , -142.97299 , -124.29318 , -123.499016, -103.57509 ,
        -81.41246 ,  -75.778984,  -67.58444 ,  -64.26604 ,  -63.435867,
        -53.794296,  -47.21119 ,  -39.84661 ,  -34.155067,  -18.853592,
         -9.774348,   25.318073,   34.12614 ,   61.343266,   76.40645 ,
         98.77449 ,  134.31041 ,  150.33508 ,  151.9047  ,  152.27783 ,
        170.37514 ,  186.46767 ,  207.88503 ,  211.9901  ,  224.78212 ,
        229.69115 ,  238.16728 ,  362.2559  ,  446.6527  ,  573.1921  ],
      dtype=float32), np.int64(4): array([-2.78376007e+02, -2.57589844e+02, -2.43326462e+02, -2.23126770e+02,
       -2.05721252e+02, -1.60718369e+02, -1.57585403e+02, -1.54615189e+02,
       -1.50316772e+02, -1.477340